In [1]:
import numpy as np
import pandas as pd
from sklearn.kernel_approximation import RBFSampler
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error, roc_curve, classification_report,auc)
from sklearn.preprocessing import LabelEncoder

In [2]:
# First read the parquet files without specifying names
traindata = pd.read_parquet('C:/Users/Liberty/OneDrive/Desktop/iot project/dataset/UNSW_NB15_training-set.parquet')
testdata = pd.read_parquet('C:/Users/Liberty/OneDrive/Desktop/iot project/dataset/UNSW_NB15_testing-set.parquet')

In [12]:
df = pd.DataFrame(traindata)

# Display the shape and column names
print("Data Shape:", df.shape)
print("Columns:")
print(df.columns.tolist())

# Display information about the data types and a sample of the data
print("\nData Types:")
print(df.dtypes)
print("\nFirst few rows:")
print(df.head())

Data Shape: (175341, 36)
Columns:
['dur', 'proto', 'service', 'state', 'spkts', 'dpkts', 'sbytes', 'dbytes', 'rate', 'sload', 'dload', 'sloss', 'dloss', 'sinpkt', 'dinpkt', 'sjit', 'djit', 'swin', 'stcpb', 'dtcpb', 'dwin', 'tcprtt', 'synack', 'ackdat', 'smean', 'dmean', 'trans_depth', 'response_body_len', 'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'is_ftp_login', 'ct_ftp_cmd', 'ct_flw_http_mthd', 'is_sm_ips_ports', 'attack_cat', 'label']

Data Types:
dur                   float32
proto                category
service              category
state                category
spkts                   int16
dpkts                   int16
sbytes                  int32
dbytes                  int32
rate                  float32
sload                 float32
dload                 float32
sloss                   int16
dloss                   int16
sinpkt                float32
dinpkt                float32
sjit                  float32
djit                  float32
swin                    int16
stcpb   

In [ ]:
df = pd.DataFrame(testdata)

# Display the shape and column names
print("Data Shape:", df.shape)
print("Columns:")
print(df.columns.tolist())

# Display infdf = pd.DataFrame(testdata)

# Display the shape and column names
print(f"Data Shape: {df.shape}")
print("Columns:")
print(df.columns.tolist())

# Display information about the data types and a sample of the data
print("\nData Types:")
print(df.dtypes)
print("\nFirst few rows:")
print(df.head())
print("\nData Types:")
print(df.dtypes)
print("\nFirst few rows:")
print(df.head())

Data Shape: (82332, 36)
Columns:
['dur', 'proto', 'service', 'state', 'spkts', 'dpkts', 'sbytes', 'dbytes', 'rate', 'sload', 'dload', 'sloss', 'dloss', 'sinpkt', 'dinpkt', 'sjit', 'djit', 'swin', 'stcpb', 'dtcpb', 'dwin', 'tcprtt', 'synack', 'ackdat', 'smean', 'dmean', 'trans_depth', 'response_body_len', 'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'is_ftp_login', 'ct_ftp_cmd', 'ct_flw_http_mthd', 'is_sm_ips_ports', 'attack_cat', 'label']

Data Types:
dur                   float32
proto                category
service              category
state                category
spkts                   int16
dpkts                   int16
sbytes                  int32
dbytes                  int32
rate                  float32
sload                 float32
dload                 float32
sloss                   int16
dloss                   int16
sinpkt                float32
dinpkt                float32
sjit                  float32
djit                  float32
swin                    int16
stcpb    

In [5]:
# If you need to rename the columns, you can do it after reading:
column_names = ['dur', 'proto', 'service', 'state', 'spkts', 'dpkts', 'sbytes',
                'dbytes', 'rate', 'sload', 'dload', 'sloss', 'dloss', 'sinpkt',
                'dinpkt', 'sjit', 'djit', 'swin', 'stcpb', 'dtcpb', 'dwin', 'tcprtt',
                'synack', 'ackdat', 'smean', 'dmean', 'trans_depth',
                'response_body_len', 'ct_src_dport_ltm', 'ct_dst_sport_ltm',
                'is_ftp_login', 'ct_ftp_cmd', 'ct_flw_http_mthd', 'is_sm_ips_ports',
                'attack_cat', 'label']

traindata.columns = column_names
testdata.columns = column_names

In [6]:
for column in traindata.columns:
    if traindata[column].dtype == type(object):
        le = LabelEncoder()
        traindata[column] = le.fit_transform(traindata[column])

In [7]:
for column in testdata.columns:
    if testdata[column].dtype == type(object):
        le = LabelEncoder()
        testdata[column] = le.fit_transform(testdata[column])

In [9]:
print(traindata.shape)


(175341, 36)


In [10]:
X1 = traindata.iloc[:, 1:35]  
Y1 = traindata.iloc[:, 35]
X2 = testdata.iloc[:, 1:35]
Y2 = testdata.iloc[:, 35]

In [14]:
# Create one-hot encoded features for the categorical columns
X1_numeric = pd.get_dummies(X1)
X2_numeric = pd.get_dummies(X2)

# Make sure both training and testing have the same columns
X1_numeric, X2_numeric = X1_numeric.align(X2_numeric, join='outer', axis=1, fill_value=0)

# Normalize your numeric data
scaler = Normalizer().fit(X1_numeric)
trainX = scaler.transform(X1_numeric)

scaler = Normalizer().fit(X2_numeric)
testT = scaler.transform(X2_numeric)

print("Transformed training shape:", trainX.shape)
print("Transformed testing shape:", testT.shape)

Transformed training shape: (175341, 197)
Transformed testing shape: (82332, 197)


In [15]:
traindata = np.array(trainX)
trainlabel = np.array(Y1)

testdata = np.array(testT)
testlabel = np.array(Y2)

In [17]:
model = DecisionTreeClassifier()
model.fit(traindata, trainlabel)
print(model)
# make predictions
expected = testlabel
predicted = model.predict(testdata)
np.savetxt('res/predictedDT.txt', predicted, fmt='%01d')
np.savetxt('res/expectedDT.txt', expected, fmt='%01d')
# summarize the fit of the model
accuracy = accuracy_score(expected, predicted)
recall = recall_score(expected, predicted, average="binary")
precision = precision_score(expected, predicted , average="binary")
f1 = f1_score(expected, predicted , average="binary")

DecisionTreeClassifier()


In [19]:
cm = metrics.confusion_matrix(expected, predicted)
print("Confusion Matrix:")
print(cm)

# Correct computation:
# TPR (Recall/Sensitivity) = TP / (TP + FN)
tpr = float(cm[1][1]) / np.sum(cm[1])
# FPR = FP / (FP + TN)
fpr = float(cm[0][1]) / np.sum(cm[0])

print("True Positive Rate (Recall): %.3f" % tpr)
print("False Positive Rate: %.3f" % fpr)
print("Accuracy")
print("%.3f" %accuracy)
print("precision")
print("%.3f" %precision)
print("recall")
print("%.3f" %recall)
print("f-score")
print("%.3f" %f1)
print("fpr")
print("%.3f" %fpr)
print("tpr")
print("%.3f" %tpr)
print("***************************************************************")

Confusion Matrix:
[[24911 12089]
 [ 3454 41878]]
True Positive Rate (Recall): 0.924
False Positive Rate: 0.327
Accuracy
0.811
precision
0.776
recall
0.924
f-score
0.843
fpr
0.327
tpr
0.924
***************************************************************
